In [1]:
import torch
from PIL import Image
# from diffusers import StableDiffusionImg2ImgPipeline 
from diffusers import AutoPipelineForImage2Image
import webcolors  
from io import BytesIO
assert torch.cuda.is_available()

torch.cuda.empty_cache()
pipeline = AutoPipelineForImage2Image.from_pretrained("kandinsky-community/kandinsky-2-2-decoder", torch_dtype=torch.float16, variant="fp16", use_safetensors=True).to("cuda")
# pipe = StableDiffusionImg2ImgPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", torch_dtype=torch.float16).to("cuda")
pipeline.enable_model_cpu_offload()
pipeline.enable_xformers_memory_efficient_attention()

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in {"jpg", "jpeg", "png", "gif"}

def closest_color(rgb):
    diffs = {}
    for color_hex, color_name in webcolors.CSS3_HEX_TO_NAMES.items():
        r, g, b = webcolors.hex_to_rgb(color_hex)
        diffs[sum([(r - rgb[0])**2,
                   (g - rgb[1])**2,
                   (b - rgb[2])**2])] = color_name
        
    return diffs[min(diffs.keys())]

def convert_to_rgb(hexcode):
    try:
        # Hex renk kodundan gereksiz karakterleri kaldırın ve doğrudan RGB'ye çevirin
        hexcode = hexcode.replace('"', '').lstrip("#") 
        rgb = tuple(int(hexcode[i:i+2], 16) for i in (0, 2, 4))
        return rgb
    except ValueError:
        # Hata durumunda varsayılan bir değer döndürün (örneğin siyah renk: (0, 0, 0))
        return (0, 0, 0)

def generate_img(image, prompt, hexcode):
    if allowed_file(image):
        # Convert and process the image
        prompt += closest_color(convert_to_rgb(hexcode))
        print(prompt)
        image = Image.open(image)
        image = image.convert("RGB")
        buf = BytesIO()
        image.save(buf, format='PNG')
        output = pipeline(prompt=prompt, image=image, generator=generator).images[0]
        
        return output  # Image nesnesini dön



seed = torch.randint(0, 1000000, (1,)).item()
generator = torch.Generator("cuda").manual_seed(seed)

aiimg_hexcode = "#7FFF33"
prompt = "A coffee cup on a table, oil painting, 4k, detailed, background color: "
image = "coffee_cup1.png"

generate_img(image, prompt, aiimg_hexcode)

c:\Users\user\Documents\GitHub\stable-diffusion-api\venv\Lib\site-packages\diffusers\pipelines\pipeline_utils.py:1480: FutureWarning: You are trying to load the model files of the `variant=fp16`, but no such modeling files are available.The default model files: {'movq/diffusion_pytorch_model.safetensors', 'movq/diffusion_pytorch_model.bin', 'unet/diffusion_pytorch_model.safetensors', 'unet/diffusion_pytorch_model.bin'} will be loaded instead. Make sure to not load from `variant=fp16`if such variant modeling files are not available. Doing so will lead to an error in v0.22.0 as defaulting to non-variantmodeling files is deprecated.
  deprecate("no variant default", "0.22.0", deprecation_message, standard_warn=False)
movq\diffusion_pytorch_model.safetensors not found


OSError: Could not found the necessary `safetensors` weights in {'movq/diffusion_pytorch_model.safetensors', 'movq/diffusion_pytorch_model.bin', 'unet/diffusion_pytorch_model.safetensors', 'unet/diffusion_pytorch_model.bin'} (variant=fp16)

In [ ]:
# output